<a href="https://colab.research.google.com/github/AmirHoseein99/Depression-Engine/blob/master/VAD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
[1]*10**10

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_train.shape

(3601, 500, 1)

In [ ]:
y_train.shape

(3601,)

In [ ]:
import os
import glob
import pickle
import librosa
import sklearn
import logging
import itertools
import numpy as np
import pandas as pd
import librosa.display
import tensorflow as tf
from keras import layers  
from tensorflow import keras
from matplotlib import pyplot as plt
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
! pip install git+https://github.com/artemmavrin/focal-loss.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_audio_path = r'/content/drive/MyDrive/Train'
test_audio_path = r'/content/drive/MyDrive/Test'
valid_audio_path = r'/content/drive/MyDrive/Valid'

In [ ]:
def readFeatures (path):
  all_Feature = {}
  all_labels = {}
  all_files = glob.glob(path + "/*.csv")
  for filename in all_files:
      feature = pd.read_csv(filename)
      features = feature.iloc[:, 10:34]
      label = feature.iloc[:, 1:2]
      # feature
      all_Feature[int(filename.split("/")[5].split("_")[0])] = features.to_numpy()
      #label
      all_labels[int(filename.split("/")[5].split("_")[0])] = label.to_numpy()
  return all_Feature, all_labels

In [ ]:
X_train, y_train = readFeatures(train_audio_path)
X_valid, y_valid = readFeatures(valid_audio_path)

In [ ]:
X_train_path = r"/content/drive/MyDrive/train/MFCC/X_train.pkl"
X_train_delta_path = r"/content/drive/MyDrive/train/MFCC/X_train_delta.pkl"
y_train_path = r"/content/drive/MyDrive/train/y_train_dict.pkl"
X_valid_path = r"/content/drive/MyDrive/valid/MFCC/X_valid"
X_valid_delta_path = r"/content/drive/MyDrive/valid/MFCC/X_valid_delta"
y_valid_path = r"/content/drive/MyDrive/valid/y_valid.pkl"

In [ ]:
X_train_dict  = pd.read_pickle(X_train_path)
X_train_delta_dict  = pd.read_pickle(X_train_delta_path)
y_train_dict = pd.read_pickle(y_train_path)
X_valid_dict = pd.read_pickle(X_valid_path)
X_valid_delta_dict = pd.read_pickle(X_valid_delta_path)
y_valid_dict = pd.read_pickle(y_valid_path)

In [ ]:
X_train_dict = []
y_train_dict = []
y_valid_dict = []
X_valid_dict = []
X_valid_delta_dict = []
X_train_delta_dict = []

In [ ]:
delta_X_train = []
delta_X_valid = []

In [ ]:
X_train = []
delta_X_train = []
y_train = []
for key in X_train_dict.keys():
  X_train.append(X_train_dict[key])
  delta_X_train.append(X_train_delta_dict[key])
  y_train.append(y_train_dict[key])
X_train = [item for sublist in X_train for item in sublist]
delta_X_train = [item for sublist in delta_X_train for item in sublist]
y_train = [item for sublist in y_train for item in sublist]
X_train = np.array(X_train)
delta_X_train = np.array(delta_X_train)
y_train = np.array(y_train)

In [ ]:
X_valid = []
delta_X_valid = []
y_valid = []
for key in y_valid_dict.keys():
  X_valid.append(X_valid_dict[key])
  delta_X_valid.append(X_valid_delta_dict[key])
  y_valid.append(y_valid_dict[key])
X_valid = [item for sublist in X_valid for item in sublist]
delta_X_valid = [item for sublist in delta_X_valid for item in sublist]
y_valid = [item for sublist in y_valid for item in sublist]
X_valid = np.array(X_valid)
delta_X_valid = np.array(delta_X_valid)
y_valid = np.array(y_valid)

In [ ]:
X_train = np.append(X_train, delta_X_train, axis = 1)
X_valid = np.append(X_valid, delta_X_valid, axis = 1)

In [ ]:
X_valid.shape

(3606088, 52)

In [ ]:
X_train = np.array(list(X_train.values()))
y_train = np.array(list(y_train.values()))
X_train = [item for sublist in X_train for item in sublist]
y_train = [item for sublist in y_train for item in sublist]
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_valid = np.array(list(X_valid.values()))
y_valid = np.array(list(y_valid.values()))
X_valid = [item for sublist in X_valid for item in sublist]
y_valid = [item for sublist in y_valid for item in sublist]
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1]))

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=4, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=2, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=1, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    conv4 = keras.layers.Conv1D(filters=64, kernel_size=1, padding="same")(conv2)
    conv4 = keras.layers.BatchNormalization()(conv4)
    conv4 = keras.layers.ReLU()(conv4)

    gap = keras.layers.GlobalAveragePooling1D()(conv4)

    Dense1 = keras.layers.Dense(32)(gap)
    Dense1 = keras.layers.ReLU()(Dense1)

    output_layer = keras.layers.Dense(1, activation="sigmoid")(Dense1)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from keras import layers

input_spectrogram = layers.Input((None, 52))

Reshape= layers.Reshape((-1, 52, 1))(input_spectrogram)

Conv1 = layers.Conv2D(filters=64, kernel_size=[4, 4], padding="same", )(Reshape)
Conv1 = layers.BatchNormalization()(Conv1)
Conv1 = layers.ReLU()(Conv1)


Conv2 = layers.Conv2D(filters=64, kernel_size=[4, 4], padding="same")(Conv1)
Conv2 = layers.BatchNormalization()(Conv2)
Conv2 = layers.ReLU()(Conv2)


Reshape = layers.Reshape((-1, Conv2.shape[-2] * Conv2.shape[-1]))(Conv2)



Gru1 = layers.GRU(units=64, return_sequences=True, dropout=.5)(Reshape)
Gru2 = layers.GRU(units=64, return_sequences=True, dropout=.5)(Gru1)

gap = keras.layers.GlobalAveragePooling1D()(Gru2)

hybrid_output = layers.Dense(units=1, activation="sigmoid")(gap)

hybrid_model = keras.Model(input_spectrogram, hybrid_output)

In [ ]:
epochs = 500
batch_size = 1024
# import focal_loss

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "/content/drive/MyDrive/hybrid_model/weights.{epoch:02d}-{val_loss:.2f}.hdf5.h5", monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
# hybrid_model.compile(
#     optimizer="adam",
#     loss=keras.losses.BinaryCrossentropy(),
#     metrics=["accuracy", keras.metrics.AUC()],
# )
tf.get_logger().setLevel(logging.ERROR)
# history = hybrid_model.fit(
#     X_train,
#     y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     callbacks=callbacks,
#     validation_data=(X_valid, y_valid),
#     verbose=1,
# )

In [ ]:
new_model = keras.models.load_model("/content/drive/MyDrive/hybrid_model/weights.33-0.30.hdf5.h5")

In [ ]:
history = new_model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_valid, y_valid),
    verbose=1,
    initial_epoch=33
)

Epoch 34/500
9459/9459 [==============================] - 566s 59ms/step - loss: 0.2909 - accuracy: 0.8784 - auc_1: 0.9430 - val_loss: 0.3017 - val_accuracy: 0.8761 - val_auc_1: 0.9410 - lr: 0.0010
Epoch 35/500
9459/9459 [==============================] - 550s 58ms/step - loss: 0.2907 - accuracy: 0.8785 - auc_1: 0.9431 - val_loss: 0.3009 - val_accuracy: 0.8757 - val_auc_1: 0.9413 - lr: 0.0010
Epoch 36/500
9459/9459 [==============================] - 552s 58ms/step - loss: 0.2907 - accuracy: 0.8785 - auc_1: 0.9431 - val_loss: 0.3041 - val_accuracy: 0.8721 - val_auc_1: 0.9414 - lr: 0.0010
Epoch 37/500
9459/9459 [==============================] - 551s 58ms/step - loss: 0.2906 - accuracy: 0.8785 - auc_1: 0.9431 - val_loss: 0.3042 - val_accuracy: 0.8739 - val_auc_1: 0.9408 - lr: 0.0010
Epoch 38/500
9459/9459 [==============================] - 548s 58ms/step - loss: 0.2906 - accuracy: 0.8784 - auc_1: 0.9431 - val_loss: 0.3045 - val_accuracy: 0.8730 - val_auc_1: 0.9408 - lr: 0.0010
Epoch 39/5

In [ ]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
from matplotlib import pyplot as plt
import sklearn
import glob
import pickle
import itertools
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
# delta_1 = open("/content/drive/MyDrive/train_delta_1.pkl", "rb")
# train_delta_1 = pickle.load(delta_1)
# delta_2 = open("/content/drive/MyDrive/train_delta_2.pkl", "rb")
# train_delta_2 = pickle.load(delta_2)
mfccs = open("/content/drive/MyDrive/train_mfcc.pkl", "rb")
train_set = pickle.load(mfccs)
y = open("/content/drive/MyDrive/y_train.pkl", "rb")
y_train = pickle.load(y)

In [ ]:
mfccs = open("/content/drive/MyDrive/X_valid.pkl", "rb")
X_valid = pickle.load(mfccs)
# delta_1 = open("/content/drive/MyDrive/valid_delta_1.pkl", "rb")
# valid_delta_1 = pickle.load(delta_1)
# delta_2 = open("/content/drive/MyDrive/valid_delta_2.pkl", "rb")
# valid_delta_2 = pickle.load(delta_2)
y = open("/content/drive/MyDrive/y_valid.pkl", "rb")
y_valid = pickle.load(y)

In [ ]:
X_valid = X_valid[:, :24]

In [ ]:
X_valid.shape

(3606088, 24)

In [ ]:
train_set = np.append(train_set, train_delta_1, axis=1)
# train_set = np.append(train_set, y_train, axis=1)

In [ ]:
train_set.shape

(9685240, 48)

In [ ]:
train_set = pd.DataFrame(train_set)

In [ ]:
train_class_0 = train_set[train_set.iloc[:,-1] ==0]
train_class_1 = train_set[train_set.iloc[:,-1] ==1]

In [ ]:
train_class_1.shape[0] + (0.4 * train_class_1.shape[0])

5025342.0

In [ ]:
under_class_0 = train_class_0.sample(train_class_1.shape[0])

In [ ]:
train_set = pd.concat([under_class_0, train_class_1])

In [ ]:
train_set = train_set.to_numpy()

In [ ]:
train_set.shape

(7179060, 25)

In [ ]:
X_valid = np.append(X_valid, y_valid, axis=1)

In [ ]:
valid_set.shape

In [ ]:
# train_delta_1 = np.append(train_delta_1, train_delta_2, axis=1)
# valid_delta_1 = np.append(valid_delta_1, valid_delta_2, axis=1)

In [ ]:
# train_ds = np.append(train_mfccs, train_delta_1, axis=1)
# valid_ds = np.append(valid_mfccs, valid_delta_1, axis=1)

In [ ]:
# valid_set = tf.data.Dataset.from_tensor_slices((valid_set[:, :-1], valid_set[:, -1]))
train_set = tf.data.Dataset.from_tensor_slices((train_set[:, :-1], train_set[:, -1]))

In [ ]:
X_valid = X_valid.reshape(-1, 1, 24, 1)
train_set = train_set.reshape(-1, 1, 24, 1)

In [ ]:
X_valid.shape

(3606088, 48, 1, 1)

In [ ]:
X_valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
train_set = tf.data.Dataset.from_tensor_slices((train_set, y_train))

In [ ]:
batch_size = 512
window_length = 100
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, train=False):
  # Batch all datasets.
  # ds = ds.window(window_length, shift=50, drop_remainder=True)
  # ds = ds.flat_map(lambda window: window.batch(window_length))

  if train == True :
    ds = ds.shuffle(100)

  ds = ds.batch(batch_size)

  return ds.prefetch(AUTOTUNE)

In [ ]:
train_set = prepare(train_set, train=True)
X_valid = prepare(X_valid)

In [ ]:
from keras import layers
input = layers.Input((None, 24), name="input")
    # Expand the dimension to use 2D CNN.
x = layers.Reshape((-1, 24, 1), name="expand_dim")(input)
    # Convolution layer 1
x = layers.Conv2D(filters=16, kernel_size=[4, 4], padding="same", kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = layers.BatchNormalization(name="conv_1_bn")(x)
x = layers.ReLU(name="conv_1_relu")(x)
x = layers.MaxPooling2D((2,2), padding="same")(x)
    # Convolution layer 2
# x = layers.Conv2D(filters=64, kernel_size=[4, 4], padding="same",)(x)
# x = layers.BatchNormalization(name="conv_2_bn")(x)
# x = layers.ReLU(name="conv_2_relu")(x)
# x = layers.MaxPooling2D((2,2), padding="same")(x)
    # Reshape the resulted volume to feed the RNNs layers
x = layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)
    # RNN layers
# x = layers.GRU(units=128, return_sequences = True)(x)
# x = layers.Dropout(rate=0.5)(x)
x = layers.GRU(units=50, kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = layers.Dropout(rate=0.5)(x)    
# Dense layer
# x = layers.Dense(units=128, name="dense", kernel_initializer="he_normal")(x)
# x = layers.ReLU(name="dense_relu")(x)
# x = layers.Dropout(rate=0.5)(x)
x = layers.Dense(units=10, name="dense2", kernel_initializer="he_normal",  kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = layers.ReLU(name="dense_2_relu")(x)
    # Classification layer
output = layers.Dense(units=1, activation="sigmoid")(x)
    # Model
model = keras.Model(input, output)

In [ ]:
input = keras.layers.Input((None, 48))
x = keras.layers.Reshape((-1, 48, 1))(input)
x = keras.layers.Conv2D(32, 4, padding="same")(x)
x = keras.layers.BatchNormalization()(x) 
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(64, 4, padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)
x = keras.layers.SimpleRNN(64)(x)
x = keras.layers.Dropout(.5)(x)
x = keras.layers.Dense(64)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.BatchNormalization()(x)
output = keras.layers.Dense(units=1, activation="sigmoid")(x)
model = keras.Model(input, output)

In [ ]:
model = keras.models.Sequential([     
    keras.layers.Conv2D(32, 4, activation="relu", padding="same",
    input_shape=[1, 24, 1]),    
    keras.layers.MaxPooling2D((2,2), padding='same'),                       
    # keras.layers.Conv2D(64, 4, activation="relu", padding="same"),
    # keras.layers.MaxPooling2D((2,2), padding='same'),  
    # keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    # keras.layers.MaxPooling2D((2,2), padding='same'),  
    # keras.layers.BatchNormalization(),
    # keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    # keras.layers.MaxPooling2D((2,2), padding='same'),  
    # keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    # keras.layers.LSTM(64, input_shape=[]),
    keras.layers.Dense(60, activation="relu"),
    # keras.layers.SimpleRNN(64),
    keras.layers.Dropout(.5),
    # keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 24, 32)         544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 12, 32)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 12, 64)         32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 6, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 3, 128)        0

In [ ]:
def exponential_decay(lr0, s):
  def exponential_decay_fn(step):
    return lr0 * 0.1**(step / s)
  return exponential_decay_fn
exponential_decay_fn = exponential_decay(lr0=0.05, s=20)

In [ ]:
opt = keras.optimizers.Adam(clipvalue=1.0)
import focal_loss
model.compile(
            loss=focal_loss.BinaryFocalLoss(gamma=2),
            optimizer=opt,
            metrics=["accuracy", keras.metrics.AUC()]
)

In [ ]:
es_callback = keras.callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=True)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
history = model.fit(train_set, epochs=20, validation_data=X_valid , verbose = 1, callbacks=[es_callback], batch_size=512)

Epoch 1/20
18917/18917 [==============================] - 461s 24ms/step - loss: 0.0874 - accuracy: 0.8641 - auc_3: 0.9280 - val_loss: 1.8109 - val_accuracy: 0.6233 - val_auc_3: 0.7971
Epoch 2/20
18917/18917 [==============================] - 459s 24ms/step - loss: 0.0863 - accuracy: 0.8658 - auc_3: 0.9300 - val_loss: 1.5161 - val_accuracy: 0.6233 - val_auc_3: 0.8589
Epoch 3/20
18917/18917 [==============================] - 480s 25ms/step - loss: 0.0860 - accuracy: 0.8663 - auc_3: 0.9305 - val_loss: 1.8197 - val_accuracy: 0.6233 - val_auc_3: 0.7865
Epoch 4/20
18917/18917 [==============================] - 459s 24ms/step - loss: 0.0860 - accuracy: 0.8664 - auc_3: 0.9307 - val_loss: 1.6453 - val_accuracy: 0.6233 - val_auc_3: 0.8285
Epoch 5/20
18917/18917 [==============================] - 404s 21ms/step - loss: 0.0858 - accuracy: 0.8665 - auc_3: 0.9310 - val_loss: 1.7618 - val_accuracy: 0.6233 - val_auc_3: 0.8010
Epoch 6/20
18917/18917 [==============================] - 458s 24ms/step - 